In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect



### Extract & Transform for Calendar data

In [2]:
# create path for calendar file and read
calendar_file = "Resources/calendar.csv"

calendar_df = pd.read_csv(calendar_file, encoding="utf8")
# print first 5 rows to check df
calendar_df.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [3]:
# exploring
calendar_new_df = calendar_df[['listing_id', 'date','available', 'price']].copy()
calendar_new_df

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN
...,...,...,...,...
1393565,10208623,2016-12-29,f,NaN
1393566,10208623,2016-12-30,f,NaN
1393567,10208623,2016-12-31,f,NaN
1393568,10208623,2017-01-01,f,NaN


In [4]:
# replace NaN values for $0 so that it can be used. 
calendar_new_df['price'] = calendar_new_df['price'] .fillna('$0')
clean_calendar_df = calendar_new_df


# remove duplicate if there are any
clean_calendar_df.drop_duplicates(inplace = True)


# converting the string to datetime format
clean_calendar_df['date'] = pd.to_datetime(clean_calendar_df['date'])

# converting the string to float format
clean_calendar_df['price'] = clean_calendar_df['price'].replace({'\$': '', ',': ''}, regex= True).astype(float)


# print to review
clean_calendar_df.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,85.0
1,241032,2016-01-05,t,85.0
2,241032,2016-01-06,f,0.0
3,241032,2016-01-07,f,0.0
4,241032,2016-01-08,f,0.0


In [5]:
# check types
clean_calendar_df.dtypes

listing_id             int64
date          datetime64[ns]
available             object
price                float64
dtype: object

### Extract & Transform for listings data

In [6]:
# create path for listings file and read
listings_file = "Resources/listings.csv"

listings_df = pd.read_csv(listings_file, encoding="utf8")
listings_df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [7]:
# check all columns
listings_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [8]:
# set new df with desired columns
listings_new_df = listings_df[['id','number_of_reviews', 'review_scores_rating']].copy()
listings_new_df

,id,number_of_reviews,review_scores_rating
0,241032,207,95.0
1,953595,43,96.0
2,3308979,20,97.0
3,7421966,0,NaN
4,278830,38,92.0
...,...,...,...
3813,8101950,1,80.0
3814,8902327,2,100.0
3815,10267360,0,NaN
3816,9604740,0,NaN


In [9]:
# drop all NaN values
listings_new_df.dropna(inplace=True)
listings_new_df

,id,number_of_reviews,review_scores_rating
0,241032,207,95.0
1,953595,43,96.0
2,3308979,20,97.0
4,278830,38,92.0
5,5956968,17,95.0
...,...,...,...
3810,262764,5,92.0
3811,8578490,2,100.0
3812,3383329,73,96.0
3813,8101950,1,80.0


In [10]:
# check df types 
listings_new_df.dtypes

id                        int64
number_of_reviews         int64
review_scores_rating    float64
dtype: object

In [11]:
# create listings_host table
listings_host_df = listings_df[['id','host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'zipcode']].copy()
listings_host_df

,id,host_response_rate,host_acceptance_rate,host_is_superhost,zipcode
0,241032,96%,100%,f,98119
1,953595,98%,100%,t,98119
2,3308979,67%,100%,f,98119
3,7421966,NaN,NaN,f,98119
4,278830,100%,NaN,f,98119
...,...,...,...,...,...
3813,8101950,99%,100%,f,98107
3814,8902327,100%,100%,f,98102
3815,10267360,NaN,NaN,f,98178
3816,9604740,100%,NaN,f,98112


In [12]:
# Setting condition for keeping only Zipcodes with length of 5 digits
listings_host_df = listings_host_df[listings_host_df['zipcode'].str.len() == 5]
listings_host_df

,id,host_response_rate,host_acceptance_rate,host_is_superhost,zipcode
0,241032,96%,100%,f,98119
1,953595,98%,100%,t,98119
2,3308979,67%,100%,f,98119
3,7421966,NaN,NaN,f,98119
4,278830,100%,NaN,f,98119
...,...,...,...,...,...
3813,8101950,99%,100%,f,98107
3814,8902327,100%,100%,f,98102
3815,10267360,NaN,NaN,f,98178
3816,9604740,100%,NaN,f,98112


In [13]:
# Drop Nan values
listings_host_df = listings_host_df.dropna()
listings_host_df.head()

,id,host_response_rate,host_acceptance_rate,host_is_superhost,zipcode
0,241032,96%,100%,f,98119
1,953595,98%,100%,t,98119
2,3308979,67%,100%,f,98119
6,1909058,100%,100%,t,98119
7,856550,100%,100%,t,98119


In [14]:
# Creating type_price table
type_price_df = listings_df[["id","property_type","room_type","accommodates","square_feet","weekly_price","monthly_price"]].copy()
type_price_df.head()

,id,property_type,room_type,accommodates,square_feet,weekly_price,monthly_price
0,241032,Apartment,Entire home/apt,4,NaN,NaN,NaN
1,953595,Apartment,Entire home/apt,4,NaN,"$1,000.00","$3,000.00"
2,3308979,House,Entire home/apt,11,NaN,NaN,NaN
3,7421966,Apartment,Entire home/apt,3,NaN,$650.00,"$2,300.00"
4,278830,House,Entire home/apt,6,NaN,NaN,NaN


In [15]:
# Unit conversion (Converting strings to numeric)
# Find columns with a currency value
data_currency_cols=["weekly_price","monthly_price"]

# Remove currency signs
type_price_df[data_currency_cols]=type_price_df[data_currency_cols].replace({'\$': '', ',': ''}, regex=True)

# convert to a number
type_price_df["accommodates"] = pd.to_numeric(type_price_df["accommodates"])
type_price_df["square_feet"] = pd.to_numeric(type_price_df["square_feet"])
type_price_df["weekly_price"] = pd.to_numeric(type_price_df["weekly_price"])
type_price_df["monthly_price"] = pd.to_numeric(type_price_df["monthly_price"])

type_price_df.head()

,id,property_type,room_type,accommodates,square_feet,weekly_price,monthly_price
0,241032,Apartment,Entire home/apt,4,NaN,NaN,NaN
1,953595,Apartment,Entire home/apt,4,NaN,1000.0,3000.0
2,3308979,House,Entire home/apt,11,NaN,NaN,NaN
3,7421966,Apartment,Entire home/apt,3,NaN,650.0,2300.0
4,278830,House,Entire home/apt,6,NaN,NaN,NaN


In [16]:
# drop Nan values
type_price_df.dropna(subset=["property_type","room_type","accommodates","square_feet","weekly_price","monthly_price"], inplace=True)
type_price_df.head()

,id,property_type,room_type,accommodates,square_feet,weekly_price,monthly_price
103,1579615,House,Entire home/apt,3,485.0,590.0,1795.0
163,227636,Apartment,Entire home/apt,3,800.0,698.0,2552.0
182,1618842,Apartment,Entire home/apt,4,450.0,625.0,2000.0
212,781474,Boat,Entire home/apt,1,300.0,560.0,2195.0
234,1167507,Apartment,Entire home/apt,4,500.0,720.0,2500.0


### Connect to a local Database

In [17]:
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'airbnb_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

### Check for tables


In [18]:
insp.get_table_names()

['listings_reviews', 'listings_host', 'calendar', 'type_and_price']

### Use pandas to load csv converted DataFrame into database

In [24]:
clean_calendar_df.to_sql(name='calendar', con=engine, if_exists='append', index=False)



In [20]:
listings_new_df.to_sql(name='listings_reviews', con=engine, if_exists='append', index=False)

In [23]:
listings_host_df.to_sql(name='listings_host', con=engine, if_exists='append', index=False)

In [34]:
type_price_df.to_sql(name='type_and_price', con=engine, if_exists='append', index=False)

### Confirm Data has been added by querying the calendar table

In [25]:
pd.read_sql_query('select * from calendar', con=engine).head()

,listing_id,date,available,price
0,241032,2016-01-04,t,85.0
1,241032,2016-01-05,t,85.0
2,241032,2016-01-06,f,0.0
3,241032,2016-01-07,f,0.0
4,241032,2016-01-08,f,0.0


### Confirm Data has been added by querying the listings_review table

In [21]:
pd.read_sql_query('select * from listings_reviews', con=engine).head()

,id,number_of_reviews,review_scores_rating
0,241032,207,95.0
1,953595,43,96.0
2,3308979,20,97.0
3,7421966,0,NaN
4,278830,38,92.0


### Perform SQL joins of tables

In [22]:
sql_join2 = r"""SELECT calendar.listing_id, calendar.date, calendar.price, type_and_price.property_type, type_and_price.room_type, 
type_and_price.accommodates, type_and_price.square_feet, type_and_price.weekly_price, type_and_price.monthly_price, 
listings_reviews.number_of_reviews, listings_reviews.review_scores_rating, listings_host.host_response_rate, 
listings_host.host_acceptance_rate, listings_host.zipcode
FROM calendar
JOIN listings_host ON calendar.listing_id = listings_host.id
JOIN listings_reviews ON calendar.listing_id = listings_reviews.id
JOIN type_and_price ON calendar.listing_id = type_and_price.id"""

### Check the join table

In [29]:
pd.read_sql_query(sql_join2, con=engine)

,listing_id,date,price,property_type,room_type,accommodates,square_feet,weekly_price,monthly_price,number_of_reviews,review_scores_rating,host_response_rate,host_acceptance_rate,zipcode
0,1579615,2016-01-04,97.0,House,Entire home/apt,3,485.0,590.0,1795.0,44,98.0,100%,100%,98107
1,1579615,2016-01-04,97.0,House,Entire home/apt,3,485.0,590.0,1795.0,44,98.0,100%,100%,98107
2,1579615,2016-01-04,97.0,House,Entire home/apt,3,485.0,590.0,1795.0,44,98.0,100%,100%,98107
3,1579615,2016-01-04,97.0,House,Entire home/apt,3,485.0,590.0,1795.0,44,98.0,100%,100%,98107
4,1579615,2016-01-05,97.0,House,Entire home/apt,3,485.0,590.0,1795.0,44,98.0,100%,100%,98107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55475,262764,2017-01-01,0.0,House,Entire home/apt,5,1200.0,1120.0,5500.0,5,92.0,100%,100%,98103
55476,262764,2017-01-02,0.0,House,Entire home/apt,5,1200.0,1120.0,5500.0,5,92.0,100%,100%,98103
55477,262764,2017-01-02,0.0,House,Entire home/apt,5,1200.0,1120.0,5500.0,5,92.0,100%,100%,98103
55478,262764,2017-01-02,0.0,House,Entire home/apt,5,1200.0,1120.0,5500.0,5,92.0,100%,100%,98103
